<a href="https://colab.research.google.com/github/cqschlortt/Create-an-Oracle-for-Shor-s-Algorithm/blob/main/Shor's_Algorithm_Oracle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ORACLE FOR SHOR’S ALGORITHM- using 2n+3 qubits aproach



Using the aproach described in "Circuit for Shor's algorithm using $2n+3$ qubits, S. Beauregard, arXiv:quant-ph/0205095" we implemented an oracle such that for all $a,N\in\mathbb{Z}_+$ such that $a<N$ outputs:

\begin{align}
U|c>_1|y>_n=
\begin{cases}
|c>_1|ay\;\mod(N)>_n & |c>=|1>\text{ and } y<N\\
|c>_1|y>_n & \text{otherwise}
\end{cases}
\end{align}




In [1]:
%pip install qiskit numpy
%pip install pylatexenc
%pip install qiskit_aer
from qiskit.circuit import QuantumCircuit, QuantumRegister, AncillaRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Statevector, Operator

from qiskit_aer import AerSimulator, Aer
from qiskit.compiler import transpile

import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=22a594cf66a042b8f7551f55423ac53fddc998b210e3d66ffadf37fa9edbac9d
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 92.2 MB/s eta 0:00:00


##Imporant Variables Used Throughout

###N_val:
The number we want to factor in Shor's Algorithm

###a_val
A value smaller than N_val with which we want to test if it is a factor of N_val

###n
The number of bits required to represent N_val

##Create Adder Gate

The following functions are applied to create the adder gate and the reverse adder gate needed to implement the modular adder gate

###get_add_gate

Parameters: n and a_val (as presented above)

Objective: Add the classical value a_val to a quantum value b (represtented by a quantum register of size n) in the Fourier space, keep the first n qubits the same (as they contain a_val) and change the second n qubits to hold the quantum Fourier transform of (a_val$ + b) \mod 2^n$.

Output: A gate which can be added to a circuit with at least 2n qubits and adds the value represented by the first n qubits to that represented by the second n qubits

Method: Use Draper's quantum addition algortihm to add the qubits in a representing a_val to those in b (and their sum is left in b)

###conditional_phase_shift

Parameters: qc is the circuit to apply to, control_qubit is the control, target_qubit is the target, k is an integer which determines which phase shift to look at

Objective: Calculate the phase shift angle and apply phase shift to specific qubit

Output: A gate with a single control and single target which applies a phase shift to the target if the control holds a 1

###get_inv_add_gate

Parameters: a_val and n (as described above)

Obective: Get the reverse to the adder gate which allows for subtraction (not in the Fourier space)

Output: Puts |b-a_val> in the register containing b if $b\geq $ a_val and |$2^{n+1}-$(a_val - b)> otherwise

In [2]:
# Addition Circuit
# Uses Conditional Phase Shift gates to calculate the QFT(a+b)

def get_add_gate(n,a_val=None):

  quantum_register_a= QuantumRegister(size=n, name='a')
  quantum_register_b= QuantumRegister(size=n, name='b')

  qc = QuantumCircuit(quantum_register_a,quantum_register_b)

  # Create a QFT circuit with n qubits
  qft_b_circuit = QFT(num_qubits=n).to_gate()

  # Add the QFT circuit to the main circuit
  #qc.append(qft_b_circuit, quantum_register_b)

  # Define the conditional phase shift function
  def conditional_phase_shift(qc: QuantumCircuit, control_qubit, target_qubit, k: int):

    # Calculate the phase angle: theta = 2π / 2^k
    theta = (2 * np.pi) / (2**k)

      # The Qiskit `cp` (controlled-phase) gate directly implements the matrix:
      # [[1, 0, 0, 0],
      #  [0, 1, 0, 0],
      #  [0, 0, 1, 0],
      #  [0, 0, 0, e^(i*theta)]]
    qc.cp(theta, control_qubit, target_qubit)
  for j in range(n+1):

    for i in range(n-j):
      requested_k =i+1
      #print(j,i,n-(j+1),n-(i+j+1),requested_k)
      conditional_phase_shift(qc, quantum_register_a[n-(i+j+1)], quantum_register_b[n-(j+1)], k=requested_k)

  label = f"ADD(a,n={n})" if a_val is None else f"ADD({a_val})"
  return qc.to_gate(label=label)

  #print("Circuit")
  # Draw the circuit
  #print(qc.draw(output='text', idle_wires=False))



def get_inv_add_gate(n,a_val=None):

  quantum_register_a= QuantumRegister(size=n, name='a')
  quantum_register_b= QuantumRegister(size=n, name='b')

  qcinv = QuantumCircuit(quantum_register_a,quantum_register_b)

  fix_add_gate = get_add_gate(n,a_val)
  inv_add_gate = fix_add_gate.inverse()
  qcinv.append(inv_add_gate, quantum_register_a[:] + quantum_register_b[:])
  label = f"inv ADD(a,n={n})" if a_val is None else f"inv ADD({a_val})"
  return qcinv.to_gate(label=label)



##Modular Adder Gate

The following function creates the modular adder gate

###get_add_a_mod_N_gate

Parameters: n, a_val, N_val (as described abover)

Objective: Add a_val to some value b mod N_val

Output: A gate which can be applied to a circuit which adds a_val + b mod N_val in the Fourier space.

Method: Convert a_val and N_val into binary strings, create a controlled get_add_gate for a_val (controlled by the size 2 quantum register c with target b) to get $\phi$(a_val + b) in the b register.  Then a get_inv_add_gate is applied (with N_val as what is being added) in order to get $\phi$(a_val + b - N_val) in the b register.  The inverse QFT is then applied to the b register and a controlled x gate is applied with the most significant bit of b as the control and an ancilla (d) as the target.  If a_val + b < N_val, this flips the value in d to a 1.  We then apply QFT to return b to containing $\phi$(a_val + b - N_val), and apply a controlled get_add_gate (with N_val) where d is the control and b is the target.  If d flipped to 1 because a_val + b < N_val, then this adds back N_val, and does nothing otherwise.

Now register b contains $\phi$(a_val + b) if a_val + b < N_val and $\phi$(a_val + b - N) otherwise, so it contains $\phi$(a + b) mod N.

The last steps just return  the ancilla to its original value by applying get_inv_add_gate (with a_val) with controls as the register c to subtract a_val, applying inverse QFT on b and then an x gate on the most significant digit of b.  If a_val + b < N_val, this will mean the value in this qubit is now 1, and 0 otherwise.  Then a controlled x gate is applied to d (where d is the target and the most signifigant qubit of b is the control) which flips d only if the most significant qubit of b is 1.  Thus, if d was flipped once before, it is restored to 0, and if it was not flipped, it remains 0.  

Finally, another x gate is applied to the most signifcant qubit of b, then QFT is applied to b, then get_add_gate (with a_val) with controls as the register c are applied to restore b to contain $\phi$(a + b) mod N.

In [3]:
#Create the function for gate "Add a mod N"

def get_add_a_mod_N_gate(n,a_val=None,N_val=None):
  quantum_register_a= QuantumRegister(size=n, name='a')
  quantum_register_b= QuantumRegister(size=n, name='b')
  quantum_register_c= QuantumRegister(size=2, name='c')
  ancilla_register_d= AncillaRegister(size=1, name='d')
  quantum_register_N= QuantumRegister(size=n, name='N')

  qc4 = QuantumCircuit(quantum_register_c, quantum_register_a, quantum_register_b, ancilla_register_d, quantum_register_N)

  #convert a and N

  for i in range(n):
      if (a_val >> i) & 1:
        qc4.x(quantum_register_a[i])
      if (N_val >> i) & 1:
        qc4.x(quantum_register_N[i])

  # Create a QFT circuit with n qubits
  qft_b_circuit = QFT(num_qubits=n).to_gate()


  # Add the QFT circuit to the main circuit
  #qc4.append(qft_b_circuit, quantum_register_b)

  # We create a control gate for ADD(a) controlled by c.
  num_c=2
  # Get the ADD(a) gate by calling the function
  add_a_gate = get_add_gate(n=n, a_val=a_val)
  # Call the .control() method on the gate
  controlled_add_gate = add_a_gate.control(num_ctrl_qubits=num_c,)

  # Add the ADD(a) gate controlled by c to the main circuit
  all_qubits_for_controlled_add_a = list(quantum_register_c[:]) + list(quantum_register_a[:]) + list(quantum_register_b[:])
  qc4.append(controlled_add_gate, all_qubits_for_controlled_add_a)


  # Get the inv ADD(N) gate by calling the function
  inv_add_N_gate = get_inv_add_gate(n=n, a_val=N_val)
  #Add the inverse ADD(N)
  all_qubits_for_inv_add_N = ( quantum_register_N[:] + quantum_register_b[:] )
  qc4.append(inv_add_N_gate, all_qubits_for_inv_add_N)

  # Create an inverse QFT circuit with n qubits
  qft_b_circuit_inverse = qft_b_circuit.inverse()
  qft_b_circuit_inverse.name = "Inv QFT"

  # Add the inverse QFT circuit to the main circuit
  qc4.append(qft_b_circuit_inverse, quantum_register_b)

  # Add a control gate controled by d
  qc4.cx(quantum_register_b[n-1], ancilla_register_d[0])

  # Add the QFT circuit to the main circuit
  qft_b_gate = qft_b_circuit # Convert QFT to gate
  qc4.append(qft_b_gate, quantum_register_b)

  # Get the ADD(N) gate by calling the function
  add_N_gate = get_add_gate(n=n, a_val=N_val)
  #Add the ADD(N)
  all_qubits_for_add_N = ( quantum_register_N[:] + quantum_register_b[:] )
  qc4.append(add_N_gate, all_qubits_for_add_N)

  # Get the inv ADD(a) gate by calling the function
  inv_add_a_gate = get_inv_add_gate(n=n, a_val=a_val)
  #Add the inverse ADD(a)
  all_qubits_for_inv_add_a = ( quantum_register_a[:] + quantum_register_b[:] )

  # We create a control gate for inv ADD(a) controlled by c.
  controlled_inv_add_a_gate = inv_add_a_gate.control(num_ctrl_qubits=num_c,)

  # Add the ADD(a) gate controlled by c to the main circuit
  all_qubits_for_controlled_inv_add_a = list(quantum_register_c[:]) + list(quantum_register_a[:]) + list(quantum_register_b[:])
  qc4.append(controlled_inv_add_a_gate, all_qubits_for_controlled_inv_add_a)

  # Add the inverse QFT circuit to the main circuit
  qc4.append(qft_b_circuit_inverse, quantum_register_b)

  # Add a x gate to the most significant bit of b
  qc4.x(quantum_register_b[n-1])

  # Add a control gate controled by x
  qc4.cx(quantum_register_b[n-1], ancilla_register_d[0])

  # Add a x gate to the most significant bit of b
  qc4.x(quantum_register_b[n-1])

  # Add the QFT circuit to the main circuit
  qc4.append(qft_b_gate, quantum_register_b)

  # Add the ADD(a) gate controlled by c to the main circuit
  qc4.append(controlled_add_gate, all_qubits_for_controlled_add_a)
  label = f"ADD(a,mod{N_val} n={n})" if a_val is None else f"ADD({a_val}) mod {N_val}"
  return qc4.to_gate(label=label)



##Controlled Multiplier Gate

###get_cmult_a_mod_N_gate

Parameters: n, a_val, N_val (as described above)

Objective: Create a controlled gate which takes the qubits |c>|y>|b> (where c is a single control qubit, y is a number represented by n qubits and b is as above), and find |(b+a_val*y)modN> in the b register

Output: A gate applied to a circuit so that the b register will hold |(b+a_val*y)modN> if c held 1 and will be unchanged otherwise

Method: Apply QFT to the register holding b to get into the Fourier space.  Then for each i in $[0, n-1] \cap \mathbb{Z}$, apply get_add_a_mod_N_gate with a_val = $2^i$a_val (mod N_val) with the controls c and y_i and the target b.  Then apply inverse QFT so that we leave the Fourier space.  If c is 0, then it will always be 0, and no controlled addition will be performed.  If c is 1, then addition will be performed when y_i is 1 (or when that bit is 'contributing')

In [4]:
#Create the function for multiplier gate
def get_cmult_a_mod_N_gate(n,a_val=None,N_val=None):

  quantum_register_y= QuantumRegister(size=n, name='y')
  quantum_register_b= QuantumRegister(size=n, name='b')
  quantum_register_c= QuantumRegister(size=1, name='c')
  quantum_register_a= QuantumRegister(size=n, name='a')
  quantum_register_N= QuantumRegister(size=n, name='N')
  ancilla_register_d = AncillaRegister(size=1, name='d')
  a_val_times_power = a_val


  qc4 = QuantumCircuit(quantum_register_c, quantum_register_y, quantum_register_b, ancilla_register_d, quantum_register_N, quantum_register_a)



  #quantum_register_a= AncillaRegister(size=n, name='a')
  #quantum_register_N= AncillaRegister(size=n, name='N')

   #Get QFT and QFT Inverse
  qft_b_circuit = QFT(num_qubits=n)                # This is a QuantumCircuit
  qft_b_gate = qft_b_circuit.to_gate()             # Convert to Gate
  qft_b_gate_inverse = qft_b_circuit.inverse().to_gate()  # Invert THEN convert

   #Add QFT to circuit
  qc4.append(qft_b_gate, quantum_register_b)

  #Apply controlled a mod N gates for each 2^0a, 2^1a,...
  for i in range(n):
    if i>0:
        a_val_times_power =a_val_times_power*2 % N_val
   # else:
    #    a_val_times_power =a_val
   #Get controlled a mod N gate
    add_a_mod_N_gate=get_add_a_mod_N_gate(n=n ,a_val=a_val_times_power,N_val=N_val)
    all_qubits_for_add_a = [quantum_register_c[0], quantum_register_y[i]] + list(quantum_register_a[:]) + list(quantum_register_b[:]) + list(ancilla_register_d[:]) + list(quantum_register_N[:])
    qc4.append(add_a_mod_N_gate, all_qubits_for_add_a)

  #Apply inverse QFT
  qc4.append(qft_b_gate_inverse, quantum_register_b)


  label = f"CMULT({a_val}) mod {N_val}"
  return qc4.to_gate(label=label)
  return qc4.to_gate()


###swap
To be used in the construction of the oracle

Parameters: n (as above)

Objective: Swap the values between two different registers

Method: Apply controlled x gates in order to switch values

In [5]:
def swap(n):
  quantum_register_a= QuantumRegister(size=n, name='a')
  quantum_register_b= QuantumRegister(size=n, name='b')

  qc = QuantumCircuit(quantum_register_a,quantum_register_b)

  for i in range(n):
    qc.cx(quantum_register_b[i], quantum_register_a[i])
    qc.cx(quantum_register_a[i], quantum_register_b[i])
    qc.cx(quantum_register_b[i], quantum_register_a[i])

  return qc.to_gate()

## Oracle for Shor's Algorithm

### get_U_a_gate

Parameters: n, a_val, N_val as above

Objective: If the control is 1, return a_val*y mod N, and remain unchanged otherwise

Method: Apply get_cmult_a_mod_N_gate followed by the swap gate and then apply inverse get_cmult_a_mod_N_gate to a inverse to produce desired result

In [6]:
def get_U_a_gate(n, a_val=None, N_val=None):
  quantum_register_c= QuantumRegister(size=1, name='c')
  quantum_register_x= QuantumRegister(size=n, name='x')
  quantum_register_z= QuantumRegister(size=n, name='zero')

 # Temporary/internal registers
  quantum_register_a = QuantumRegister(n, name='a')
  quantum_register_N = QuantumRegister(n, name='N')
  ancilla_register_d = AncillaRegister(1, name='d')

  qc4 = QuantumCircuit(quantum_register_c, quantum_register_x, quantum_register_z, quantum_register_a, quantum_register_N, ancilla_register_d)

  #Get controlled swap
  swap_gate=swap(n)
  controlled_swap_gate = swap_gate.control(num_ctrl_qubits=1)

  #Apply cmult gate to circuit
  cmult_gate=get_cmult_a_mod_N_gate(n=n,a_val=a_val,N_val=N_val)
  qc4.append(cmult_gate,
               quantum_register_c[:] +
               quantum_register_x[:] +
               quantum_register_a[:] +
               quantum_register_z[:] +
               ancilla_register_d[:] +
               quantum_register_N[:])

  #Apply controlled swap
  qc4.append(controlled_swap_gate, quantum_register_c[:] + quantum_register_x[:] + quantum_register_z[:])

  #Find a inverse
  a_inv=pow(a_val,-1,N_val)


  #Apply cmult inverse gate acting on a inverse
  temp = get_cmult_a_mod_N_gate(n, a_val, N_val)
  cmult_gate_a_inv=temp.inverse()
  qc4.append(cmult_gate_a_inv,
               quantum_register_c[:] +
               quantum_register_x[:] +
               quantum_register_a[:] +
               quantum_register_z[:] +
               ancilla_register_d[:] +
               quantum_register_N[:])

  # Return the gate
  label = f"U_a({a_val}) mod {N_val}"
  return qc4.to_gate(label=label)
